In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sn
import scipy.io as sio
import os

In [10]:
def csi_to_features(csi_data):
    features=[]
    csi_data=np.array(csi_data)
    array_length=csi_data.shape[0] #or .size
    
    #use sd scope of 20
    # sd_sampling_range = 20
    # for i in range(0,array_length-sd_sampling_range):
    #     sd_value= np.std(csi_data[i:i+sd_sampling_range])
    #     features.append(sd_value)
    # return features
        
    #It seems to be some odd operation that is not sd
    #It is root mean square RMS ok
    sampling_range=10
    mean_val=csi_data.mean()
    for i in range(0,array_length-sampling_range):
        vals=[]
        for j in range(i,i+sampling_range):
            val=math.pow(csi_data[j]-mean_val,2)
            vals.append(val)
        val_append=math.sqrt(sum(vals)/sampling_range)
        features.append(val_append)
    return features

In [12]:
#File reading to dataframe for training
quartile="Q1"
section="01"
axis="Horizontal"
file_dir = f"../CSI_Data_27122023/{axis}/{quartile}_{section}.csv"
file_dir = f"csi_data.csv"
data=pd.read_csv(file_dir)
#get only csi data (which is 26th column aka. -1)
# df=data.iloc[:,25]
data
# df=data.iloc[:,-1]
# df

,CSI_DATA,AP,A4:CF:12:8F:B7:74,-39,11,0,0.1,0.2,0.3,0.4,...,4,1,28159,0.10,30,0.11,0.12,0.205512,128,[30 -32 1 0 0 0 0 0 0 0 0 0 -2 -26 -1 -25 0 -27 1 -27 3 -26 4 -24 4 -25 3 -23 5 -24 4 -23 4 -24 4 -23 4 -23 5 -22 4 -21 3 -23 4 -20 4 -21 3 -21 2 -21 2 -21 2 -21 2 -20 1 -21 0 -22 1 -22 0 0 1 -21 -1 -21 -1 -21 -1 -21 -1 -21 -2 -21 -3 -21 -2 -20 -2 -19 -3 -19 -4 -20 -3 -18 -4 -19 -5 -20 -4 -19 -6 -19 -5 -20 -5 -20 -5 -19 -5 -19 -5 -19 -4 -19 -3 -19 -3 -20 -3 -19 -2 -19 0 0 0 0 0 0 0 0 0 0 ]
0,CSI_DATA,AP,A4:CF:12:8F:B7:74,-40,11,0,0,0,0,0,...,4,1,68993,0,28,0,0,0.246247,128,[28 -64 1 0 0 0 0 0 0 0 0 0 -18 16 -19 15 -20 ...
1,CSI_DATA,AP,A4:CF:12:8F:B7:74,-40,11,0,0,0,0,0,...,4,1,71761,0,157,0,0,0.253314,128,[-99 -48 9 0 0 0 0 0 0 0 0 0 2 21 2 21 1 21 1 ...
2,CSI_DATA,AP,A4:CF:12:8F:B7:74,-39,11,0,0,0,0,0,...,4,1,90658,0,135,0,0,0.267852,128,[-121 112 8 0 0 0 0 0 0 0 0 0 22 -19 23 -18 24...
3,CSI_DATA,AP,A4:CF:12:8F:B7:74,-49,11,1,7,1,1,1,...,4,1,99718,0,390,0,0,0.276719,384,[-122 -31 23 0 0 0 0 0 0 0 0 0 -7 20 -7 20 -8 ...
4,CSI_DATA,AP,A4:CF:12:8F:B7:74,-49,11,1,7,1,1,1,...,4,1,102425,0,390,0,0,0.283331,384,[-122 -31 23 0 0 0 0 0 0 0 0 0 20 2 20 3 20 4 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,CSI_DATA,AP,A4:CF:12:8F:B7:74,-49,11,1,6,1,1,1,...,4,1,6353378,0,84,0,0,6.530290,384,[84 -64 4 0 0 0 0 0 0 0 0 0 2 22 0 21 0 22 -1 ...
450,CSI_DATA,AP,A4:CF:12:8F:B7:74,-49,11,1,7,1,1,1,...,4,1,6359265,0,84,0,0,6.536490,384,[84 -64 4 0 0 0 0 0 0 0 0 0 20 -3 20 -2 19 -1 ...
451,CSI_DATA,AP,A4:CF:12:8F:B7:74,-50,11,1,7,1,1,1,...,4,1,6369290,0,84,0,0,6.546280,384,[84 -64 4 0 0 0 0 0 0 0 0 0 12 18 12 18 11 18 ...
452,CSI_DATA,AP,A4:CF:12:8F:B7:74,-49,11,1,7,1,1,1,...,4,1,6379390,0,84,0,0,6.556270,384,[84 -64 4 0 0 0 0 0 0 0 0 0 -6 -20 -5 -20 -4 -...


In [5]:
#reform df
csi_data=[]
#equivalent to 2000 samples which is 2 seconds (crop 2 seconds out from start and end)
crop_row_start=1000;
crop_row_end=len(df)-1000;
for i in range(crop_row_start,crop_row_end):
    s=df[i].replace("[","").replace("]","").strip()
    ss=s.split(" ")
    csi_values=[float(val) for val in ss]
    csi_data.append(csi_values)
csi_data=np.array(csi_data)
csi_data_df=pd.DataFrame(csi_data)
csi_data


array([], dtype=float64)